# Jupyter Notebook to Reproduce the Race Condition between Cert-Manager and Service CA Operator

## Create a Kubernetes Cluster with Kind

In [ ]:
!kind create cluster

## Install Cert-Manager with Selfsigned Cluster Issuer

In [ ]:
!kubectl apply -f https://github.com/cert-manager/cert-manager/releases/download/v1.13.1/cert-manager.yaml

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: cert-manager.io/v1
kind: ClusterIssuer
metadata:
  name: selfsigned-cluster-issuer
spec:
  selfSigned: {}
EOF

## Install Service CA Operator to Kubernetes

In [ ]:
!kubectl apply -f https://raw.githubusercontent.com/ykoer/kubernetes-service-ca/main/service-ca-deployment.yaml

## Create a Service with a Service-Serving-Cert

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: v1
kind: Service
metadata:
  name: test-service
  namespace: default
  annotations:
    service.beta.openshift.io/serving-cert-secret-name: test-service-tls
spec:
  ports:
  - name: https
    port: 8443
  selector:
    app: test-service
EOF

In [ ]:
!kubectl get secret test-service-tls -oyaml -n default

## Create a Certificate Resource

In [ ]:
%%bash
kubectl create -f - <<EOF
apiVersion: cert-manager.io/v1
kind: Certificate
metadata:
  name: certificaterequest-loop-reproducer
  namespace: default
spec:
  issuerRef:
    group: cert-manager.io
    kind: ClusterIssuer
    name: selfsigned-cluster-issuer
  secretName: test-service-tls # <--- reference a secret with the ownerReference
  commonName: test-service.mydomain.com
  privateKey:
    algorithm: RSA
    size: 3072
    rotationPolicy: Always
    encoding: PKCS1
EOF

In [ ]:
!kubectl get crs -n default

In [ ]:
!kubectl delete certificate certificaterequest-loop-reproducer -n default